In [ ]:
import numpy as np
import numpy.random as nrand
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
from itertools import combinations
from matplotlib.ticker import FormatStrFormatter
from scipy.optimize import curve_fit
import itertools
from sklearn.linear_model import LinearRegression
import uncertainties as unc
import uncertainties.unumpy as unp
from multiprocessing import Pool

In [ ]:
N_list = [5,10,15]
model_list = ['NK','RMF','Polynomial']
metric_list = ['N_max','epi','r_s','open_ratio']

In [ ]:
with open('extrapolation_result.pkl','rb') as f:
    all_df = pickle.load(f)

In [ ]:
ruggedness_measure_dict = {'N_max':'$N_{max}$','epi':'$F_{rse}$','r_s':'$r/s$','blocked_ratio':'$F_{bp}$'}

In [ ]:
# Plotting parameters

para_dict = {
    5:{
        'N_max':{'xlim':[-10,10],'ylim':[0,200]},
        'epi':{'xlim':[-0.25,0.25],'ylim':[0,200]},
        'r_s':{'xlim':[-1.5,1.5],'ylim':[0,200]},
        'blocked_ratio':{'xlim':[-0.25,0.25],'ylim':[0,200]},
    },
    10:{
        'N_max':{'xlim':[-50,100],'ylim':[0,200]},
        'epi':{'xlim':[-0.05,0.15],'ylim':[0,200]},
        'r_s':{'xlim':[-1,2.5],'ylim':[0,200]},
        'blocked_ratio':{'xlim':[-0.1,0.15],'ylim':[0,200]},
    },
    15:{
        'N_max':{'xlim':[-250,1200],'ylim':[0,200]},
        'epi':{'xlim':[-0.03,0.16],'ylim':[0,200]},
        'r_s':{'xlim':[-1,3],'ylim':[0,200]},
        'blocked_ratio':{'xlim':[-0.1,0.15],'ylim':[0,200]},
    }
}

title_dict = {
    'model': '{} model',
    'replication': '{} replicates',
    'ruggedness': 'Ruggedness level {}',
    'measurement_std': 'Estimation error sd {:.3}'
}

In [ ]:
# N_site: number of variable site of landscape; parameter: parameter for data stratification 
# N_site: 5, 10, 15; parameter: 'model', 'replication', 'ruggedness', 'measurement_std'
N_site=10
parameter='replication'

cv_cutoff = 0.5
show_all_A_B = False # If use all available alpha and beta value even though some data might 
                     # fail the extrapolation. Default is False.

denominator = all_df['averaged_ruggedness']
fig,axes = plt.subplots(4,4,figsize=[15,15])

for j,metric in enumerate(['N_max','epi','r_s','blocked_ratio']):
    for i,rep in enumerate([3,4,5,6]):
        ax = axes[j,i]
        xlim = para_dict[N_site][metric]['xlim']
        idx_all = (all_df['metric'] == metric) & (all_df['N_site'] == N_site) & (all_df[parameter] == rep)
        if rep == 3:
            idx = (all_df["convex_concave"] == 'concave') & idx_all
        else:
            idx = (np.abs(all_df["prediction_std"]/denominator) <= cv_cutoff) & idx_all
        for index,color,label in zip(['A','B','C'],['b','g','r'],['α','β','δ']):
            if show_all_A_B and index != 'C':
                hist_df = all_df[index][idx_all]
            else:
                hist_df = all_df[index][idx]
            ax.hist(hist_df, bins = 50,range=(xlim[0],xlim[1]),alpha=0.5, color = color, label=label)

            ax.annotate(s=None,xy=(np.mean(hist_df),220), xytext=(np.mean(hist_df),250),
                         arrowprops={'arrowstyle': '->', 'color': color}, va='center',size=20)

        ax.set_xlim(xlim)
        ax.set_ylim(0,250)
        ax.text(0.98, 0.5, f'$ac$={np.sum(idx)/np.sum(idx_all):.3f}', horizontalalignment='right',
                verticalalignment='top', transform=ax.transAxes, fontsize=15)
        if j == 0:
            ax.set_title(title_dict[parameter].format(rep),fontsize=15)
        if i == 0:
            ax.set_ylabel(f'Frequency',fontsize = 15)
            ax.set_xlabel(f'{ruggedness_measure_dict[metric]} estimation error', fontsize = 15)
        if (i == 0) & (j == 0):
            ax.legend(prop={'size': 15})

plt.tight_layout()
plt.show()

In [ ]:
# N_site: number of variable site of landscape; parameter: parameter for data stratification 
# N_site: 5, 10, 15; parameter: 'model', 'replication', 'ruggedness', 'measurement_std'

N_site = 10
parameter = 'model'

cv_cutoff = 0.5
show_all_A_B = False # If use all available alpha and beta value even though some data might 
                     # fail the extrapolation. Default is False.

denominator = all_df['averaged_ruggedness']
fig,axes = plt.subplots(4,3,figsize=[11,15])

for j,metric in enumerate(['N_max','epi','r_s','blocked_ratio']):
    for i,model in enumerate(['NK','RMF','Polynomial']):
        ax = axes[j,i]
        xlim = para_dict[N_site][metric]['xlim']
        idx_all = (all_df['metric'] == metric) & (all_df['N_site'] == N_site) & (all_df[parameter] == model)
        idx = ((all_df["convex_concave"] == 'concave') | \
              (np.abs(all_df["prediction_std"]/denominator) <= cv_cutoff)) & \
              idx_all
        
        for index,color,label in zip(['A','B','C'],['b','g','r'],['α','β','δ']):
            if show_all_A_B and index != 'C':
                hist_df = all_df[index][idx_all]
            else:
                hist_df = all_df[index][idx]

            ax.hist(hist_df, bins = 50,range=(xlim[0],xlim[1]),alpha=0.5, color = color, label=label)
            ax.annotate(s=None,xy=(np.mean(hist_df),270), xytext=(np.mean(hist_df),300),
                         arrowprops={'arrowstyle': '->', 'color': color}, va='center',size=20)

        ax.set_xlim(xlim)
        ax.set_ylim(0,300)
        ax.text(0.98, 0.5, f'$ac$={np.sum(idx)/np.sum(idx_all):.3f}', horizontalalignment='right',
                verticalalignment='top', transform=ax.transAxes, fontsize=15)
        if j == 0:
            ax.set_title(f'{model} model',fontsize=15)
        if i == 0:
            ax.set_ylabel(f'Frequency',fontsize = 15)
            ax.set_xlabel(f'{ruggedness_measure_dict[metric]} estimation error', fontsize = 15)
        if (i == 0) & (j == 0):
            ax.legend(prop={'size': 15})
            
plt.tight_layout()
plt.show()

In [ ]:
# N_site: number of variable site of landscape; parameter: parameter for data stratification 
# N_site: 5, 10, 15; parameter: 'model', 'replication', 'ruggedness', 'measurement_std'

N_site = 10
parameter = 'ruggedness'

cv_cutoff = 0.5
show_all_A_B = False # If use all available alpha and beta value even though some data might 
                     # fail the extrapolation. Default is False.

denominator = all_df['averaged_ruggedness']
fig,axes = plt.subplots(4,3,figsize=[11,15])

for j,metric in enumerate(['N_max','epi','r_s','blocked_ratio']):
    for i,ruggedness in enumerate([1,2,3]):
        ax = axes[j,i]
        xlim = para_dict[N_site][metric]['xlim']
        idx_all = (all_df['metric'] == metric) & (all_df['N_site'] == N_site) & (all_df[parameter] == ruggedness)
        idx = ((all_df["convex_concave"] == 'concave') | \
              (np.abs(all_df["prediction_std"]/denominator) <= cv_cutoff)) & \
              idx_all
        
        for index,color,label in zip(['A','B','C'],['b','g','r'],['α','β','δ']):
            if show_all_A_B and index != 'C':
                hist_df = all_df[index][idx_all]
            else:
                hist_df = all_df[index][idx]

            ax.hist(hist_df, bins = 50,range=(xlim[0],xlim[1]),alpha=0.5, color = color, label=label)
            ax.annotate(s=None,xy=(np.mean(hist_df),270), xytext=(np.mean(hist_df),300),
                         arrowprops={'arrowstyle': '->', 'color': color}, va='center',size=20)

        ax.set_xlim(xlim)
        ax.set_ylim(0,300)
        ax.text(0.98, 0.5, f'$ac$={np.sum(idx)/np.sum(idx_all):.3f}', horizontalalignment='right',
                verticalalignment='top', transform=ax.transAxes, fontsize=15)
        if j == 0:
            ax.set_title(f'Ruggedness level {ruggedness}',fontsize=15)
        if i == 0:
            ax.set_ylabel(f'Frequency',fontsize = 15)
            ax.set_xlabel(f'{ruggedness_measure_dict[metric]} estimation error', fontsize = 15)
        if (i == 0) & (j == 0):
            ax.legend(prop={'size': 15})
            
plt.tight_layout()
plt.show()

In [ ]:
# N_site: number of variable site of landscape; parameter: parameter for data stratification 
# N_site: 5, 10, 15; parameter: 'model', 'replication', 'ruggedness', 'measurement_std'

N_site = 10
parameter = 'measurement_std'

cv_cutoff = 0.5
show_all_A_B = False # If use all available alpha and beta value even though some data might 
                     # fail the extrapolation. Default is False.

denominator = all_df['averaged_ruggedness']
fig,axes = plt.subplots(4,4,figsize=[15,15])

for j,metric in enumerate(['N_max','epi','r_s','blocked_ratio']):
    for i,error in enumerate([1/30,2/30,3/30,4/30]):
        ax = axes[j,i]
        xlim = para_dict[N_site][metric]['xlim']
        idx_all = (all_df['metric'] == metric) & (all_df['N_site'] == N_site) & (np.abs(all_df[parameter] - error)<1e-5)
        idx = ((all_df["convex_concave"] == 'concave') | \
              (np.abs(all_df["prediction_std"]/denominator) <= cv_cutoff)) & \
              idx_all
        
        for index,color,label in zip(['A','B','C'],['b','g','r'],['α','β','δ']):
            if show_all_A_B and index != 'C':
                hist_df = all_df[index][idx_all]
            else:
                hist_df = all_df[index][idx]

            ax.hist(hist_df, bins = 50,range=(xlim[0],xlim[1]),alpha=0.5, color = color, label=label)
            ax.annotate(s=None,xy=(np.mean(hist_df),270), xytext=(np.mean(hist_df),300),
                         arrowprops={'arrowstyle': '->', 'color': color}, va='center',size=20)

        ax.set_xlim(xlim)
        ax.set_ylim(0,300)
        ax.text(0.98, 0.5, f'$ac$={np.sum(idx)/np.sum(idx_all):.3f}', horizontalalignment='right',
                verticalalignment='top', transform=ax.transAxes, fontsize=15)
        if j == 0:
            ax.set_title(f'Estimation error sd {error:.3}',fontsize=15)
        if i == 0:
            ax.set_ylabel(f'Frequency',fontsize = 15)
            ax.set_xlabel(f'{ruggedness_measure_dict[metric]} estimation error', fontsize = 15)
        if (i == 0) & (j == 0):
            ax.legend(prop={'size': 15})
            
plt.tight_layout()
plt.show()